# _Multiple_ Linear Regression (MLR) Model

## The Model

In the population we have <ins>scalar</ins> random variables, $[Y,X_1,\ldots,X_{k},e]$, that fulfill the following relationship

$$
Y=\beta_0+\beta_1 X_1+\ldots+\beta_k X_k+e\text{,}
$$

where $E[e|X_1,\ldots,X_{k}]=0$, there are no exact linear relationships among the set of regressors (covariates, confounders, independent variables, predictors, etc.) $[X_1,\ldots,X_{k}]$, and var$(e|X_1,\ldots,X_{k})<+\infty$. The scalar random variable, $Y$, is called the outcome, or the dependent variable.



<p style='text-align: right;'> <a href="https://en.wikipedia.org/wiki/Conditional_expectation" style="color: #cc0000">Conditional Expectation</a></p>

<p style='text-align: right;'> <a href="https://en.wikipedia.org/wiki/Conditional_variance" style="color: #cc0000">Conditional Variance</a></p>


## The Data

We observe a _random sample_ of $n$ observations taken from $[Y,X_1,\ldots,X_{k},e]$, i.e., $\{(y_i,x_{i,1},\ldots,x_{i,k}):i=1,\ldots,n\}$. Therefore one has

$$
\begin{array}
[c]{c}
y_{1}=\beta_{0}+\beta_{1}x_{1,1}+\cdots+\beta_{k}x_{1,k}+e_{1}\\
y_{2}=\beta_{0}+\beta_{1}x_{2,1}+\cdots+\beta_{k}x_{2,k}+e_{2}\\
y_{3}=\beta_{0}+\beta_{1}x_{3,1}+\cdots+\beta_{k}x_{3,k}+e_{3}\\
\vdots\\
y_{n}=\beta_{0}+\beta_{1}x_{n,1}+\cdots+\beta_{k}x_{n,k}+e_{n}
\end{array}
$$ or equivalently
$$\left[
\begin{array}
[c]{c}
y_{1}\\
y_{2}\\
y_{3}\\
\vdots\\
y_{n}
\end{array}
\right]  =\left[
\begin{array}
[c]{cccc}
1 & x_{1,1} & \cdots & x_{1,k}\\
1 & x_{2,1} & \cdots & x_{2,k}\\
1 & x_{3,1} & \cdots & x_{3,k}\\
\vdots & \vdots & \ddots & \vdots\\
1 & x_{n,1} & \cdots & x_{n,k}
\end{array}
\right]  \left[
\begin{array}
[c]{c}
\beta_{0}\\
\beta_{1}\\
\vdots\\
\beta_{k}
\end{array}
\right]  +\left[
\begin{array}
[c]{c}
e_{1}\\
e_{2}\\
e_{3}\\
\vdots\\
e_{n}
\end{array}
\right]
$$

that can be rewritten in **matrix form**  as

$$
\mathbf{y}=\mathbf{X}\mathbf{\beta}+\mathbf{e}\text{,}
$$

where $\mathbf{y}$ is a $n\times 1$ [vector](https://en.wikipedia.org/wiki/Vector_(mathematics_and_physics)), $\mathbf{X}$ is a $n\times(k+1)$ [matrix](https://en.wikipedia.org/wiki/Matrix_(mathematics)) - sometimes called the [*design matrix*](https://en.wikipedia.org/wiki/Design_matrix), $\mathbf{\beta}$ is a $(k+1)\times 1$ vector of <ins>unknown</ins> parameters, and $\mathbf{e}$ is a $n\times 1$ vector.

In [ ]:
%load_ext rpy2.ipython

In [ ]:
## removing everything from memory
%R rm(list=ls())
## turning all warnings off
%R options(warn=-1)

## installing the 'wooldridge' package if not previously installed
%R if (!require(wooldridge)) install.packages('wooldridge')

## loading the packages
%R library(wooldridge)

%R data(hprice2)

##  hprice2
##  Obs:   506

##  1. price                    median housing price, $
##  2. crime                    crimes committed per capita
##  3. nox                      nitrous oxide, parts per 100 mill.
##  4. rooms                    avg number of rooms per house
##  5. dist                     weighted dist. to 5 employ centers
##  6. radial                   accessibiliy index to radial hghwys
##  7. proptax                  property tax per $1000
##  8. stratio                  average student-teacher ratio
##  9. lowstat                  % of people 'lower status'
## 10. lprice                   log(price)
## 11. lnox                     log(nox)
## 12. lproptax                 log(proptax)

%R head(hprice2)

We can now transfer the `hprice2` R data frame into a pandas data frame and call it `py_hprice2`.

In [ ]:
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

from rpy2.robjects.conversion import localconverter

with localconverter(ro.default_converter + pandas2ri.converter):
  py_hprice2 = ro.conversion.rpy2py(ro.r('hprice2'))
py_hprice2.head(6)

Alternatively, we can access the `hprice2` pandas data frame directly from the `wooldridge` package.

In [ ]:
import wooldridge as woo
hprice2 = woo.dataWoo('hprice2')

## The Assumptions

**<span style="color:blue">Assumption MLR.1:</span>** $\mathbf{y}=\mathbf{X}\mathbf{\beta}+\mathbf{e}$.

<ins>Example</ins>: We specify the following model 

$$
\texttt{lprice}=\beta_{0}+\beta_{1}\texttt{lnox}+\beta_{2}\texttt{lproptax}+\beta_{3}\texttt{crime}+\beta_{4}\texttt{rooms}+\beta_{5}\texttt{dist}+\beta_{6}\texttt{radial}+\beta_{7}\texttt{stratio}+\beta_{8}\texttt{lowstat}+e
$$

💻 We can use ```as.formula``` to specify the model.

In [ ]:
## specifying the outcome variable (y) and regressors (X)
%R outcome <- 'lprice'
%R predictors <- c('lnox', 'lproptax', 'crime', 'rooms', 'dist', 'radial', 'stratio', 'lowstat')

## creating a specification of the linear model
%R f <- as.formula(paste(outcome,paste(predictors, collapse = ' + '),sep = ' ~ '))
%R print(f)

In [ ]:
outcome = 'lprice'
predictors = ["lnox", "lproptax", "crime", "rooms", "dist", "radial", "stratio", "lowstat"]
f = outcome + "~" + "+".join(predictors)
print(f)

**<span style="color:blue">Assumption MLR.2:</span>** $\{(y_i,x_{i,1},\ldots,x_{i,k}):i=1,\ldots,n\}$ is a random sample (independent and identically distributed - i.i.d.).

💻 ```head``` allows us to visualize the structure of the data.

In [ ]:
# R
%R head(subset(hprice2,select=c(outcome,predictors)))

In [ ]:
# Python
hprice2[[outcome] + predictors].head(5)

**<span style="color:blue">Assumption MLR.3:</span>** rank$[E(\mathbf{x}_i\mathbf{x}_i^\prime)]=k+1$, where $\mathbf{x}_i^\prime=[1,x_{i,1},\ldots,x_{i,k}]$. <p style='text-align: right;'> <a href="https://en.wikipedia.org/wiki/Rank_(linear_algebra)" style="color: #cc0000">Rank of a Matrix</a></p>
<p style='text-align: right;'> <a href="https://en.wikipedia.org/wiki/Transpose" style="color: #cc0000">Transpose</a></p> <p style='text-align: right;'> <a href="https://en.wikipedia.org/wiki/Expected_value" style="color: #cc0000">Expected Value</a></p>

💻 ```model.matrix``` creates a design matrix based on the declared predictors and includes a vector of ones by default as the first column.

In [ ]:
## asking R to print the design matrix for the chosen model
%R X <- model.matrix(f,data=hprice2)
%R dim(X)

In [ ]:
## asking Python to print the design matrix for the chose model
import patsy
y, X = patsy.dmatrices(f,hprice2,return_type='matrix')
X.shape

In [ ]:
## calculating & printing the sample counterpart of E[xx']
%R X.X.n <- t(X)%*%X/nrow(X)
%R X.X.n

In [ ]:
## calculating & printing the sample counterpart of E[xx']
import numpy as np
X_X_n = X.transpose()@X/X.shape[0]
print(X_X_n)

In [ ]:
## asking R to calculate the actual rank of the estimated E[xx']
%R qr(X.X.n)$rank

In [ ]:
## asking Python to calculate the actual rank of the estimated E[xx']
print(np.linalg.matrix_rank(X_X_n))

**<span style="color:blue">Assumption MLR.4:</span>** $E[\mathbf{e}|\mathbf{X}]=\mathbf{0}$.

$$
E[\mathbf{e}|\mathbf{X}]=\left[
\begin{array}
[c]{c}
E[e_{1}|\mathbf{X}]\\
E[e_{2}|\mathbf{X}]\\
E[e_{3}|\mathbf{X}]\\
\vdots\\
E[e_{n}|\mathbf{X}]
\end{array}
\right]  =\left[
\begin{array}
[c]{c}
0\\
0\\
0\\
\vdots\\
0
\end{array}
\right]=\mathbf{0}.  $$

**<span style="color:blue">Assumption MLR.5:</span>** var$(\mathbf{e}|\mathbf{X})=E[\mathbf{e}\mathbf{e}^{\prime}|\mathbf{X}]=\mathbf{D}$, i.e.,

$$
E[\mathbf{e}\mathbf{e}^{\prime}|\mathbf{X}]=
\begin{bmatrix}
E[e_{1}^{2}|\mathbf{X}] & 0 & \cdots & 0\\
0 & E[e_{2}^{2}|\mathbf{X}] & \cdots & 0\\
0 & 0 & \cdots & 0\\
\vdots & \vdots & \ddots & \vdots\\
0 & 0 & \cdots & E[e_{n}^{2}|\mathbf{X}]
\end{bmatrix}  = \mathbf{D}
$$

where $\vert\mathbf{D}\vert < +\infty$.

<p style='text-align: right;'> <a href="https://en.wikipedia.org/wiki/Determinant" style="color: #cc0000">Determinant</a></p>

## The *Ordinary Least Squares* (OLS) Estimator

Define the Sum of Squared Errors ($\mathrm{SSE}$) as a function of any candidate guess, $\mathbf{b}=[b_{0},b_{1},\cdots,b_{k}]^{\prime}$, for the unknown $[\beta_{0},\beta_{1},\beta_{2},\cdots,\beta_{k}]^{\prime}\equiv\mathbf{\beta}$, i.e.,

$$
\mathrm{SSE}(\mathbf{b})=\Sigma_{i=1}^{n}(y_{i}-b_{0}-b_{1}x_{1,i}-b_{2}x_{2,i}
-\cdots-b_{k}x_{k,i})^{2}=(\mathbf{y}-\mathbf{Xb})^{\prime}(\mathbf{y}
-\mathbf{Xb})
$$

By standard [matrix calculus](https://en.wikipedia.org/wiki/Matrix_calculus) one has
$$
\frac{\partial \mathrm{SSE}(\mathbf{b})}{\partial\mathbf{b}}=\left[
\begin{array}
[c]{c}
\partial \mathrm{SSE}(\mathbf{b})/\partial b_{0}\\
\partial \mathrm{SSE}(\mathbf{b})/\partial b_{1}\\
\partial \mathrm{SSE}(\mathbf{b})/\partial b_{2}\\
\vdots\\
\partial \mathrm{SSE}(\mathbf{b})/\partial b_{k}
\end{array}
\right]  =-2\mathbf{X}^{\prime}(\mathbf{y}-\mathbf{Xb})\text{,}
$$

and

$$
\frac{\partial^{2}\mathrm{SSE}(\mathbf{b})}{\partial\mathbf{b}\partial\mathbf{b}
^{\prime}}=\left[
\begin{array}
[c]{cccc}
\partial^{2}\mathrm{SSE}(\mathbf{b})/\partial b_{0}^{2} & \partial^{2}\mathrm{SSE}(\mathbf{b}
)/\partial b_{0}\partial b_{1} & \cdots & \partial^{2}\mathrm{SSE}(\mathbf{b})/\partial
b_{0}\partial b_{k}\\
\partial^{2}\mathrm{SSE}(\mathbf{b})/\partial b_{1}\partial b_{0} & \partial
^{2}\mathrm{SSE}(\mathbf{b})/\partial b_{1}^{2} & \cdots & \partial^{2}\mathrm{SSE}(\mathbf{b}
)/\partial b_{1}\partial b_{k}\\
\partial^{2}\mathrm{SSE}(\mathbf{b})/\partial b_{2}\partial b_{0} & \partial
^{2}\mathrm{SSE}(\mathbf{b})/\partial b_{2}\partial b_{1} & \cdots & \partial
^{2}\mathrm{SSE}(\mathbf{b})/\partial b_{2}\partial b_{k}\\
\vdots & \vdots & \ddots & \vdots\\
\partial^{2}\mathrm{SSE}(\mathbf{b})/\partial b_{k}\partial b_{0} & \partial
^{2}\mathrm{SSE}(\mathbf{b})/\partial b_{k}\partial b_{1} & \cdots & \partial
^{2}\mathrm{SSE}(\mathbf{b})/\partial b_{k}^{2}
\end{array}
\right]  =2\mathbf{X}^{\prime}\mathbf{X}\text{,}
$$

where $\mathbf{X}^{\prime}\mathbf{X}$ is a [positive definite matrix](https://en.wikipedia.org/wiki/Definiteness_of_a_matrix) by **<span style="color:blue">Assumption MLR.3</span>**.

In [ ]:
## printing 2X'X
%R 2*t(X)%*%X

In [ ]:
## printing 2X'X
print(2*X.transpose()@X)

Therefore $\left.  \partial \mathrm{SSE}(\mathbf{b})/\partial\mathbf{b}\right\vert _{\mathbf{b}=\widehat{\boldsymbol{\beta}}
}=\mathbf{0}$ defines a minima, i.e.,

$$\widehat{\boldsymbol{\beta}}=(\mathbf{X}
^{\prime}\mathbf{X})^{-1}\mathbf{X}^{\prime}\mathbf{y}.$$

Here we have used the
notation $\mathbf{A}^{-1}$ to denote the inverse of a matrix $\mathbf{A}$.

<p style='text-align: right;'> <a href="https://en.wikipedia.org/wiki/Euclidean_distance" style="color: #cc0000">Euclidean Distance</a></p>
<p style='text-align: right;'> <a href="https://en.wikipedia.org/wiki/Invertible_matrix" style="color: #cc0000">Inverse</a></p>
<p style='text-align: right;'> <a href="https://en.wikipedia.org/wiki/Maxima_and_minima" style="color: #cc0000">Maxima</a></p>

💻 Calculating the OLS estimator by hand first and then using the highly optimized ```lm``` command.

In [ ]:
## calculating OLS by hand
%R solve(t(X)%*%X)%*%(t(X)%*%hprice2$lprice)

In [ ]:
## calculating OLS by hand
from numpy.linalg import inv

np.set_printoptions(suppress=True)
print(inv(X.transpose()@X)@X.transpose()@y)

In [ ]:
## calculating OLS using the `lm' command in R
%R coef(lm(f,data=hprice2))

In [ ]:
## calculating OLS using the `formula' framework in statsmodels in Python
import statsmodels.formula.api as smf
ols = smf.ols(formula=f,data=hprice2).fit()
print(ols.params)

### The _Algebra_ of the OLS Estimator

💻 We now save the ```lm``` object and call it ```ols```

In [ ]:
%R ols <- lm(f,data=hprice2)

In [ ]:
ols = smf.ols(formula=f,data=hprice2).fit()

*Fitted Values*: $\widehat{\mathbf{y}}=\mathbf{X}\widehat{\boldsymbol{\beta}}$.

*Residuals*: $\widehat{\mathbf{e}}=\mathbf{y}-\mathbf{X} \widehat{\boldsymbol{\beta}}$.

In [ ]:
## OLS: printing the first 6 outcomes, fitted values, & residuals
%R head(data.frame(y=hprice2$lprice,y.hat=fitted(ols),e.hat=resid(ols)))

In [ ]:
## OLS: printing the first 6 outcomes, fitted values, & residuals
import pandas as pd
pd.DataFrame({'y':hprice2['lprice'],'y_hat':ols.fittedvalues.values,'e_hat':ols.resid}).head(6)

*Orthogonality*: $\mathbf{X}^{\prime}\widehat{\mathbf{e}}=\mathbf{0}$.

✏️ This follows from $\left.  \partial \mathrm{SSE}(\mathbf{b})/\partial\mathbf{b}\right\vert _{\mathbf{b}=\widehat{\boldsymbol{\beta}}
}=\mathbf{0}$, since $$-2\mathbf{X}^{\prime}(\mathbf{y}-\mathbf{X\widehat{\boldsymbol{\beta}}})=-2\mathbf{X}^{\prime}\widehat{\mathbf{e}}=\mathbf{0}$$

In [ ]:
## OLS: showing the orthogonality of the residuals and predictors
%R round(t(X)%*%resid(ols))

In [ ]:
## OLS: showing the orthogonality of the residuals and predictors
np.round(X.transpose()@ols.resid)

*Analysis-of-variance*:

$$
\begin{aligned}
\sum_{i=1}^{n}\left(y_{i}-\bar{y}\right)^{2}&=\sum_{i=1}^{n}\left(\widehat{y}_{i}-\bar{y}\right)^{2}+\sum_{i=1}^{n} \widehat{e}_{i}^{2},\\
\text{Total Sum of Squares}&=\text{Explained Sum of Squares} + \text{Residual Sum of Squares},\\
\left(\mathbf{y}-\boldsymbol{\iota}_{n} \bar{y}\right)^{\prime}\left(\mathbf{y}-\boldsymbol{\iota}_{n} \bar{y}\right)&=\left(\widehat{\mathbf{y}}-\boldsymbol{\iota}_{n} \bar{y}\right)^{\prime}\left(\widehat{\mathbf{y}}-\boldsymbol{\iota}_{n} \bar{y}\right)+\widehat{\mathbf{e}}^{\prime} \widehat{\mathbf{e}}.
\end{aligned}
$$

where $\boldsymbol{\iota}_n$ is a $n\times 1$ vector of ones.

In [ ]:
## OLS: total sum of squares
%R print(sum(anova(ols)[,2]))

## OLS: explained sum of squares
%R print(sum(anova(ols)[-9,2]))

## OLS: residual sum of squares
%R print(sum(anova(ols)[9,2]))

In [ ]:
## OLS: total sum of squares
print(ols.centered_tss)

## OLS: explained sum of squares
print(ols.ess)

## OLS: residual sum of squares
print(ols.ssr)

*Coefficient of Determination* (*$R^2$*):

$$
R^{2}=\frac{\sum_{i=1}^{n}\left(\widehat{y}_{i}-\bar{y}\right)^{2}}{\sum_{i=1}^{n}\left(y_{i}-\bar{y}\right)^{2}}=1-\frac{\sum_{i=1}^{n} \widehat{e}_{i}^{2}}{\sum_{i=1}^{n}\left(y_{i}-\bar{y}\right)^{2}}.
$$

✏️ More generally the $R^2$ in most models (linear and *non-linear*) is defined as $(\widehat{\text{corr}(\widehat{y},y)})^2$, i.e., the squared of the sample correlation coefficient between the true values and the fitted values.

In [ ]:
## OLS: R2 (manually)
%R print(sum(anova(ols)[-9,2])/sum(anova(ols)[,2]))

## OLS: R2 (from lm)
%R print(summary(ols)$r.squared)

In [ ]:
## OLS: R2 (manually)
print(ols.ess/ols.centered_tss)

## OLS: R2 (from lm)
ols.rsquared

*Adjusted R-squared* ($\overline{R}^2$):

$$
\overline{R}^{2}=1-\frac{(n-1) \sum_{i=1}^{n} \widehat{e}_{i}^{2}}{(n-k-1) \sum_{i=1}^{n}\left(y_{i}-\bar{y}\right)^{2}}
$$

✏️ Unlike the $R^2$ which cannot decrease as $k$ increases, $\overline{R}^2$ can either increase _or_ decrease with $k$.

In [ ]:
## OLS: adj. R2 (manually)
%R print(1-(sum(anova(ols)[9,2])/sum(anova(ols)[,2]))*(sum(anova(ols)[,1])/anova(ols)[9,1]))

## OLS: adj. R2 (from lm)
%R print(summary(ols)$adj.r.squared)

In [ ]:
## OLS: adj. R2 (manually)
print(1-((ols.nobs-1)/ols.df_resid)*(ols.ssr/ols.centered_tss))

## OLS: adj. R2 (from lm)
ols.rsquared_adj

*Leverage Values*:

The [leverage values](https://en.wikipedia.org/wiki/Leverage_(statistics)) for the design matrix $\mathbf{X}$ are the [diagonal](https://en.wikipedia.org/wiki/Main_diagonal) elements of the matrix $\mathbf{X}\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1} \mathbf{X}^{\prime}$. There are $n$ leverage values, and are typically written as $h_{i i}$ for $i=1, \ldots, n$. since

$$
\mathbf{X}\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1} \mathbf{X}^{\prime}=
\left(\begin{array}{c}{\mathbf{x}_{1}^{\prime}} \\ {\mathbf{x}_{2}^{\prime}} \\ {\vdots} \\ {\mathbf{x}_{n}^{\prime}}\end{array}\right)\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1}\left(\begin{array}{llll}{\mathbf{x}_{1}} & {\mathbf{x}_{2}} & {\cdots} & {\mathbf{x}_{n}}\end{array}\right)
$$

they are
$$
h_{i i}=\mathbf{x}_{i}^{\prime}\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1} \mathbf{x}_{i}.
$$

In [ ]:
## OLS: leverage values
%R hii <- hatvalues(ols)
%R head(hii)

In [ ]:
## OLS: leverage values
from statsmodels.stats.outliers_influence import OLSInfluence

hii = OLSInfluence(ols).hat_matrix_diag
hii[0:5,]

*Prediction Error* (leave-one-out residual or prediction residual):

$$\widetilde{e}_{i}=y_{i}-\widetilde{y}_{i},$$ 

where we use the leave-one-out predicted value for $y_i$, i.e.,

$$
\widetilde{y}_{i}=\mathbf{x}_{i}^{\prime} \widehat{\boldsymbol{\beta}}_{(-i)},$$

and

$$
\begin{aligned} \widehat{\boldsymbol{\beta}}_{(-i)} &=\left(\sum_{j \neq i} \mathbf{x}_{j} \mathbf{x}_{j}^{\prime}\right)^{-1}\left(\sum_{j \neq i} \mathbf{x}_{j} y_{j}\right) \\ &=\left(\mathbf{X}^{\prime} \mathbf{X}-\mathbf{x}_{i} \mathbf{x}_{i}^{\prime}\right)^{-1}\left(\mathbf{X}^{\prime} \mathbf{y}-\mathbf{x}_{i} y_{i}\right) \\ &=\left(\mathbf{X}_{(-i)}^{\prime} \mathbf{X}_{(-i)}\right)^{-1} \mathbf{X}_{(-i)}^{\prime} \mathbf{y}_{(-i)}. \end{aligned}
$$

Here, $\mathbf{X}_{(-i)}$ and $\mathbf{y}_{(-i)}$ are the data matrices omitting the $i$th row. The notation $\widehat{\boldsymbol{\beta}}_{(-i)}$ or $\widehat{\boldsymbol{\beta}}_{-i}$ is commonly
used to denote an estimator with the $i$th observation omitted.

✏️ There is a leave-one-out estimator for each observation, $i=1,\ldots,n$ so we have $n$ such estimators, and one can show that

$$
\widehat{\boldsymbol{\beta}}_{(-i)}=\widehat{\boldsymbol{\beta}}-\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1} \mathbf{x}_{i} \widetilde{e}_{i},
$$

and

$$
\widetilde{e}_{i}=\left(1-h_{i i}\right)^{-1} \widehat{e}_{i}.
$$

In [ ]:
## OLS: original residuals
%R e.hat <- resid(ols)

## OLS: calculating the prediction error
%R e.tilde <- resid(ols)/(1-hii)

In [ ]:
## OLS: original residuals
e_hat = ols.resid

## OLS: calculating the prediction error
e_tilde = ols.resid/(1-hii)

In [ ]:
## OLS: manually re-calculating OLS without observation 156
%R data.frame(beta.hat=coef(ols),beta.hat.i=coef(ols)-solve(t(X)%*%X)%*%X[156,]%*%e.tilde[156])

In [ ]:
## OLS: manually re-calculating OLS without observation 156
pd.DataFrame({'beta_hat':ols.params,
           'beta_hat_i':ols.params-inv(X.transpose()@X)@(X[155,]*e_tilde[155])})

*Estimation of Error Variance*:

The _unconditional_ error variance $\sigma^2=E(e^2_{i})$ can be estimated as

1. Estimator 1:

$$s^{2}=\frac{1}{n-k-1} \sum_{i=1}^{n} \widehat{e}_{i}^{2}.$$

2. Estimator 2:

$$\widehat{\sigma}^{2}=\frac{1}{n} \sum_{i=1}^{n} \widehat{e}_{i}^{2}.$$

3. Estimator 3:

$$\bar{\sigma}^{2}=\frac{1}{n} \sum_{i=1}^{n} \tilde{e}_{i}^{2}=\frac{1}{n} \sum_{i=1}^{n}\left(1-h_{i i}\right)^{-2} \widehat{e}_{i}^{2}.$$


✏️ When $k / n$ is small (typically, this occurs when $n$ is large), the estimators $\widehat{\sigma}^{2}, s^{2}$ and $\overline{\sigma}^{2}$ are likely to be similar to one another. However, if $k / n$ is large then $s^{2}$ and $\overline{\sigma}^{2}$ are generally preferred to $\widehat{\sigma}^{2}$. Consequently it is best to use one of the bias-corrected variance estimators in applications.

In [ ]:
## OLS: error variance estimators
%R s2 <- sum(e.hat^2)/(length(e.hat)-dim(X)[2])
%R sigma.hat2 <- sum(e.hat^2)/length(e.hat)
%R sigma.bar2 <- sum((e.hat/(1-hii))^2)/length(e.hat)
%R data.frame(s2=s2,sigma.hat2=sigma.hat2,sigma.bar2=sigma.bar2)

In [ ]:
## OLS: error variance estimators
s2 = ols.ssr/ols.df_resid
sigma_hat2 = ols.ssr/ols.nobs
sigma_bar2 = ((e_hat/(1-hii))**2).sum(axis=0)/ols.nobs

## Because all values are 'scalars' we need to pass them inside []
print(pd.DataFrame({'s2':[s2],'sigma_hat2':[sigma_hat2],'sigma_bar2':[sigma_bar2]}))

### The _Finite Sample_ Properties of the OLS Estimator

**<span style="color:blue">Mean of OLS:</span>** Under Assumptions MLR.1, MLR.2, MLR.3, and MLR.4 one has

$\begin{aligned} E(\widehat{\boldsymbol{\beta}} | \mathbf{X}) &=E\left(\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1} \mathbf{X}^{\prime} \mathbf{y} | \mathbf{X}\right) \\ &=\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1} \mathbf{X}^{\prime} E(\mathbf{y} | \mathbf{X}) \\ &=\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1} \mathbf{X}^{\prime} \mathbf{X} \boldsymbol{\beta} \\ &=\boldsymbol{\beta} \end{aligned}$

**<span style="color:blue">Variance of OLS:</span>** Firstly, let us use the notation $\mathbf{V}_{\widehat{\beta}} \stackrel{d e f}{=} \text{var}(\widehat{\boldsymbol{\beta}} | \mathbf{X})$ and recall from Assumption MLR.5 that var$(\mathbf{e}|\mathbf{X})=E[\mathbf{e}\mathbf{e}^{\prime}|\mathbf{X}]=\mathbf{D}$. Then under Assumptions MLR.1, MLR.2, MLR.3, MLR.4, and MLR.5 one has

$\begin{aligned} \mathbf{V}_{\widehat{\beta}} &=\text{var}(\widehat{\boldsymbol{\beta}} | \mathbf{X}) \\ &=\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1}\left(\mathbf{X}^{\prime} \mathbf{D} \mathbf{X}\right)\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1} \end{aligned}$

✏️ If one has *homoskedasticity*, i.e., $\mathbf{D}=\sigma^2\mathbf{I}_n$, then $\mathbf{V}_{\widehat{\boldsymbol{\beta}}}=\sigma^{2}\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1}$.

### Asymptotic Properties of the OLS Estimator

**<span style="color:blue">(Asymptotic) Consistency of OLS:</span>** As $n\rightarrow\infty$, under Assumptions MLR.1, MLR.2, MLR.3 and MLR.4 one has

$$
\widehat{\boldsymbol{\beta}}\stackrel{p}{\longrightarrow} \boldsymbol{\beta}
$$

<p style='text-align: right;'> <a href="https://en.wikipedia.org/wiki/Convergence_of_random_variables#Convergence_in_probability" style="color: #cc0000">Convergence in probability</a></p>

**<span style="color:blue">(Asymptotic) Distribution of OLS:</span>** As $n\rightarrow\infty$, under Assumptions MLR.1, MLR.2, MLR.3, MLR.4, and MLR.5 one has

$$\sqrt{n}(\widehat{\boldsymbol{\beta}}-\boldsymbol{\beta}) \stackrel{d}{\longrightarrow} \mathbf{N}\left(\mathbf{0}, \mathbf{V}_{\boldsymbol{\beta}}\right),$$

<p style='text-align: right;'> <a href="https://en.wikipedia.org/wiki/Convergence_of_random_variables#Convergence_in_distribution" style="color: #cc0000">Convergence in Distribution</a></p>

where

$$
\mathbf{V}_{\boldsymbol{\beta}}=E(\mathbf{x}_i\mathbf{x}_i^{\prime})^{-1}E[\mathbf{x}_{i}E(e_i^2|\mathbf{x}_i)\mathbf{x}_{i}^{\prime}]E(\mathbf{x}_i\mathbf{x}_i^{\prime})^{-1},
$$

and the notation $\mathbf{N}(\boldsymbol{\mu},\boldsymbol{\Sigma})$ denotes a [multivariate normal distribution](https://en.wikipedia.org/wiki/Multivariate_normal_distribution). $\mathbf{V}_{\boldsymbol{\beta}}$ is the asymptotic (asy.) [variance-covariance matrix](https://en.wikipedia.org/wiki/Covariance_matrix) of $\sqrt{n}(\widehat{\boldsymbol{\beta}}-\boldsymbol{\beta})$ and therefore $n^{-1}\mathbf{V}_{\boldsymbol{\beta}}$ is the asymptotic variance of $\widehat{\boldsymbol{\beta}}$ and it is generally _unknown_ and needs to be estimated.

$$
n^{-1}\mathbf{V}_{\boldsymbol{\beta}}=\left[
\begin{array}
[c]{cccc}
\text{asy. var}(\widehat{\beta}_{0}) & \text{asy. cov}(\widehat{\beta}
_{0},\widehat{\beta}_{1}) & \cdots & \text{asy. cov}(\widehat{\beta}
_{0},\widehat{\beta}_{k})\\
\text{asy. cov}(\widehat{\beta}_{1},\widehat{\beta}_{0}) & \text{asy.
var}(\widehat{\beta}_{1}) & \cdots & \text{asy. cov}(\widehat{\beta}
_{1},\widehat{\beta}_{k})\\
\text{asy. cov}(\widehat{\beta}_{2},\widehat{\beta}_{0}) & \text{asy.
cov}(\widehat{\beta}_{2},\widehat{\beta}_{1}) & \cdots & \text{asy.
cov}(\widehat{\beta}_{2},\widehat{\beta}_{k})\\
\vdots & \vdots & \ddots & \vdots\\
\text{asy. cov}(\widehat{\beta}_{k},\widehat{\beta}_{0}) & \text{asy.
cov}(\widehat{\beta}_{k},\widehat{\beta}_{1}) & \cdots & \text{asy.
var}(\widehat{\beta}_{k})
\end{array}
\right]
$$

✏️ Notice that $\mathbf{V}_{\boldsymbol{\beta}}$ is a $(k+1)\times(k+1)$ [square](https://en.wikipedia.org/wiki/Square_matrix), [symmetric](https://en.wikipedia.org/wiki/Symmetric_matrix), and [positive semi-definite](https://en.wikipedia.org/wiki/Definiteness_of_a_matrix) matrix.

<ins>(Asymptotic) OLS Variance Estimation</ins>:

Firstly notice that

$$
\begin{aligned} n\mathbf{V}_{\widehat{\boldsymbol{\beta}}} &=n\text{ var}(\widehat{\boldsymbol{\beta}} | \mathbf{X}) \\ &=\left(n^{-1}\mathbf{X}^{\prime} \mathbf{X}\right)^{-1}\left(n^{-1}\mathbf{X}^{\prime} \mathbf{D} \mathbf{X}\right)\left(n^{-1}\mathbf{X}^{\prime} \mathbf{X}\right)^{-1}, \end{aligned}
$$

and it has been shown that

$$n \mathbf{V}_{\widehat{\boldsymbol{\beta}}} \stackrel{p}{\longrightarrow} \mathbf{V}_{\boldsymbol{\beta}}$$

<p style='text-align: right;'> <a href="https://en.wikipedia.org/wiki/Convergence_of_random_variables#Convergence_in_probability" style="color: #cc0000">Convergence in probability</a></p>

🛑 Unfortunately $n\mathbf{V}_{\widehat{\boldsymbol{\beta}}}$ is _infeasible_ as matrix $\mathbf{D}$ (defined in Assumption MLR.5) is *not known*.

**<span style="color:red">HC0:</span>**

$$
\widehat{\mathbf{V}}_{\widehat{\boldsymbol{\beta}}}^{\mathrm{HC0}}=\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1}\left(\sum_{i=1}^{n} \mathbf{x}_{i} \widehat{e}_{i}^{2}\mathbf{x}_{i}^{\prime} \right)\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1}
$$

**<span style="color:red">HC1:</span>** (most common in *econometrics*)

$$
\widehat{\mathbf{V}}_{\widehat{\boldsymbol{\beta}}}^{\mathrm{HCl}}=\left(\frac{n}{n-k-1}\right)\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1}\left(\sum_{i=1}^{n} \mathbf{x}_{i} \widehat{e}_{i}^{2}\mathbf{x}_{i}^{\prime}\right)\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1}
$$

**<span style="color:red">HC2:</span>**

$$
\widehat{\mathbf{V}}_{\widehat{\boldsymbol{\beta}}}^{\mathrm{HC2}}=\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1}\left(\sum_{i=1}^{n} \mathbf{x}_{i} \left(1-h_{i i}\right)^{-1}\widehat{e}_{i}^{2} \mathbf{x}_{i}^{\prime} \right)\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1}
$$

**<span style="color:red">HC3:</span>**

$$
\widehat{\mathbf{V}}_{\widehat{\boldsymbol{\beta}}}^{\mathrm{HC3}}=\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1}\left(\sum_{i=1}^{n} \mathbf{x}_{i} \left(1-h_{i i}\right)^{-2}\widehat{e}_{i}^{2} \mathbf{x}_{i}^{\prime} \right)\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-1}
$$

💻 Unfortunately the default variance covariance matrix reported in ```lm``` corresponds to the _homoskedastic_ case, i.e., $\widehat{\sigma}^2(\mathbf{X}^{\prime}\mathbf{X})^{-1}$. All four estimators can be computed using the ```sandwich``` package.

In [ ]:
## installing the 'sandwich' package if not previously installed
%R if (!require(sandwich)) install.packages('sandwich')

## loading the packages
%R library(sandwich)

## OLS: calculating estimated asymptotic vcov matrices
#%R vcovHC(ols,type="HC0")
%R vcovHC(ols,type="HC1")
#%R vcovHC(ols,type="HC2")
#%R vcovHC(ols,type="HC3")

In [ ]:
## OLS: calculating estimated asymptotic vcov matrices
#print(ols.cov_HC0)
print(ols.cov_HC1)
#print(ols.cov_HC2)
#print(ols.cov_HC3)

<ins>(Asymptotic) Standard Errors</ins>:

These corresponds to the estimator of the standard deviation of the distribution of the OLS estimator $\widehat{\boldsymbol{\beta}}$, i.e., 

$$
s\left(\widehat{\beta}_{j}\right)=\sqrt{\widehat{\mathbf{V}}^{\mathrm{HC?}}_{\widehat{\beta}_{j}}}=\sqrt{\left[\widehat{\mathbf{V}}^{\mathrm{HC?}}_{\widehat{\boldsymbol{\beta}}}\right]_{j j}},
$$

for $?=0,1,2,3$.



💻 Computationally they simply correspond to the square root of the main diagonal elements of the $\widehat{\mathbf{V}}^{\mathrm{HC0}}_{\widehat{\boldsymbol{\beta}}}$, $\widehat{\mathbf{V}}^{\mathrm{HC1}}_{\widehat{\boldsymbol{\beta}}}$, $\widehat{\mathbf{V}}^{\mathrm{HC2}}_{\widehat{\boldsymbol{\beta}}}$, or $\widehat{\mathbf{V}}^{\mathrm{HC3}}_{\widehat{\boldsymbol{\beta}}}$.

In [ ]:
%R data.frame(se.HC0=sqrt(diag(vcovHC(ols,type="HC0"))),se.HC1=sqrt(diag(vcovHC(ols,type="HC1"))),se.HC2=sqrt(diag(vcovHC(ols,type="HC2"))),se.HC3=sqrt(diag(vcovHC(ols,type="HC3"))))

In [ ]:
pd.DataFrame({'se_HC0':ols.HC0_se,'se_HC1':ols.HC1_se,'se_HC2':ols.HC2_se,'se_HC3':ols.HC3_se})

<ins>$t$-statistic</ins>:

$$t=\frac{\widehat{\beta}_j-\beta_j}{s(\widehat{\beta}_j)}\stackrel{d}{\longrightarrow}N(0,1).$$

<p style='text-align: right;'> <a href="https://en.wikipedia.org/wiki/T-statistic" style="color: #cc0000">t-statistic</a></p>
<p style='text-align: right;'> <a href="https://en.wikipedia.org/wiki/Normal_distribution#Definition" style="color: #cc0000">Standard Normal</a></p>

💻 Most statistical packages will use the Student's $t$ distribution with $n-k-1$ degrees-of-freedom.

In [ ]:
## installing the 'lmtest' package if not previously installed
%R if (!require(lmtest)) install.packages('lmtest')

## loading the packages
%R library(lmtest)

## OLS: calculating standard t-statistics for 'significance'
%R coeftest(ols, vcov = vcovHC, type = 'HC1')

In [ ]:
smf.ols(formula=f,data=hprice2).fit(cov_type='HC1').summary().tables[1]

<ins>(Asymptotic) Confidence Interval</ins>:

The OLS estimator $\widehat{\beta}_j$ is called a _point estimator_ of the unknown slope parameter $\beta_j$. A broader concept is a _set estimator_ $\widehat{C}=[\widehat{L},\widehat{U}]$ which is an _interval estimator_ of $\beta_j$. An interval estimator $\widehat{C}$ is called a **confidence interval** when the goal is to set the coverage probability
to equal a pre-specified target such as $90 \%$ or $95 \%$. The conventional confidence interval for $\beta_j$ takes the form

$$
\widehat{C}=[\widehat{\beta}_j-c\cdot s(\widehat{\beta}_j),\widehat{\beta}_j+c\cdot s(\widehat{\beta}_j)],
$$

where $c$ equals the $1-\alpha$ [quantile](https://en.wikipedia.org/wiki/Quantile) of the distribution of $|Z|$, i.e., $c$ is equivalent to the $1-\alpha/2$ quantile of the standard normal distribution.

<p style='text-align: right;'> <a href="https://en.wikipedia.org/wiki/Confidence_interval" style="color: #cc0000">Confidence Interval</a></p>

In [ ]:
## OLS: calculating asymptotic 90% confidence interval
%R coefci(ols, level = 0.90, vcov = vcovHC, type = 'HC1')

⚠️ The `statsmodels` package in Python uses the quantiles from a *_t_*-distribution instead.

In [ ]:
smf.ols(formula=f,data=hprice2).fit(cov_type='HC1').conf_int(alpha=0.10, cols=None)

### Regression Intervals

In the linear regression model the conditional mean of $y_{i}$ given $\mathbf{x}_{i}=\mathbf{x}$ is
$$
m(\mathbf{x})=E\left(y_{i} | \mathbf{x}_{i}=\mathbf{x}\right)=\mathbf{x}^{\prime} \beta
$$
Thus an asymptotic 95% confidence interval for $m(\mathbf{x})$ is
$$
\left[\mathbf{x}^{\prime} \widehat{\boldsymbol{\beta}} \pm 1.96 \sqrt{\mathbf{x}^{\prime} \widehat{\mathbf{V}}_{\widehat{\boldsymbol{\beta}}}^{\mathrm{HC?}} \mathbf{x}}\right]
$$
for $?=0,1,2,3$.

In [ ]:
## installing the 'mgcv' package if not previously installed
%R if (!require(mgcv)) install.packages('mgcv')

## loading the packages
%R library(mgcv)

## OLS: performing OLS using the 'gam' function in 'mgcv'
%R ols.gam <- gam(f,data=hprice2)

## OLS: replacing the default homoskedastic vcov
%R ols.gam$Vp <- vcovHC(ols,type='HC1')

## OLS: creating the x_i=mean(x)
%R attach(hprice2)
%R new.dat=data.frame(lnox=mean(lnox),lproptax=mean(lproptax),crime=mean(crime),rooms=mean(rooms),dist=mean(dist),radial=mean(radial),stratio=mean(stratio),lowstat=mean(lowstat))
%R detach(hprice2)

## OLS: using the predict command to get the robust s.e.
%R reg.int <- predict(ols.gam,newdata=new.dat,se.fit=TRUE)

## OLS: manually estimating the (robust) regression interval
%R data.frame(L.hat=reg.int$fit-1.96*reg.int$se.fit,U.hat=reg.int$fit+1.96*reg.int$se.fit)

In [ ]:
import statsmodels.api as sm

## OLS: creating the x_i=mean(x)
r_ols = sm.OLS(y,X).fit(cov_type='HC1')

## OLS: creating the x_i=mean(x)
pred_mean_x = r_ols.get_prediction(X.mean(axis=0))
pd.DataFrame({'L.hat':pred_mean_x.summary_frame(alpha=0.05)['mean_ci_lower'],
              'U.hat':pred_mean_x.summary_frame(alpha=0.05)['mean_ci_upper']})


### Forecast Intervals

Suppose we are given a value of the regressor vector $\mathbf{x}_{n+1}$ for an individual outside the sample, and
we want to forecast (guess) $y_{n+1}$ for this individual. This is equivalent to forecasting $y_{n+1}$ given $\mathbf{x}_{n+1}=\mathbf{x}$, which will generally be a function of $\mathbf{x}$. A reasonable forecasting rule is the conditional mean $m(\mathbf{x})$ as it is the mean-square-minimizing forecast. A point forecast is the estimated conditional mean $\widehat{m}(\mathbf{x})=\mathbf{x}^{\prime} \widehat{\beta}$. We would also like a measure of uncertainty for the forecast.

The forecast error is $\widehat{e}_{n+1}=y_{n+1}-\widehat{m}(\mathbf{x})=e_{n+1}-\mathbf{x}^{\prime}(\widehat{\boldsymbol{\beta}}-\boldsymbol{\beta})$. As the out-of-sample error $e_{n+1}$ is independent of the in-sample estimate $\widehat{\boldsymbol{\beta}}$, this has conditional variance

$$
\begin{aligned}
E\left(\widehat{e}_{n+1}^{2} | \mathbf{x}_{n+1}=\mathbf{x}\right) &=E\left(e_{n+1}^{2}-2 \mathbf{x}^{\prime}(\widehat{\boldsymbol{\beta}}-\boldsymbol{\beta}) e_{n+1}+\mathbf{x}^{\prime}(\hat{\boldsymbol{\beta}}-\boldsymbol{\beta})(\hat{\boldsymbol{\beta}}-\boldsymbol{\beta})^{\prime} \mathbf{x} | \mathbf{x}_{n+1}=\mathbf{x}\right) \\ &=E\left(e_{n+1}^{2} | \mathbf{x}_{n+1}=\mathbf{x}\right)+\mathbf{x}^{\prime} E[(\widehat{\boldsymbol{\beta}}-\boldsymbol{\beta})(\widehat{\boldsymbol{\beta}}-\boldsymbol{\beta})^{\prime}|\mathbf{x}_{n+1}=\mathbf{x}] \mathbf{x} \\ &=\sigma^{2}(\mathbf{x})+\mathbf{x}^{\prime} \mathbf{V}_{\widehat{\boldsymbol{\beta}}} \mathbf{x} \end{aligned}
$$

Under homoskedasticity $E\left(e_{n+1}^{2} | \mathbf{x}_{n+1}\right)=\sigma^{2}$. In this case $\widehat{\sigma}^{2}+\mathbf{x}^{\prime} \widehat{\mathbf{V}}_{\widehat{\beta}}^{\mathrm{HC?}} \mathbf{x}$ is a consistent estimator of this conditional variance, so a standard error for the forecast is $\widehat{s}(\mathbf{x})=\sqrt{\widehat{\sigma}^{2}+\mathbf{x}^{\prime} \widehat{\mathbf{V}}_{\widehat{\boldsymbol{\beta}}}^{\mathrm{HC?}} \mathbf{x}}$, and the conventional 95% forecast insterval for $y_{n+1}$ uses a normal approximation and sets

$$\left[\mathbf{x}^{\prime} \widehat{\boldsymbol{\beta}} \pm 1.96 \widehat{s}(\mathbf{x})\right].$$

🛑 This interval is only valid if $e_{n+1}$ comes from a _homoskedastic_ normal distribution, i.e., $e_{n+1}\sim N(0,\sigma^2)$!

💻 We now _manually_ calculate the forecast interval for the same mean values of $\mathbf{x}$ as before.

In [ ]:
## OLS: manually estimating the forecast interval
%R s.hat <- sqrt(summary(ols)$sigma^2+reg.int$se.fit^2)
%R data.frame(L.hat=reg.int$fit-1.96*s.hat,U.hat=reg.int$fit+1.96*s.hat)

In [ ]:
## OLS: creating the x_i=mean(x)
pred_mean_x = r_ols.get_prediction(X.mean(axis=0))
pd.DataFrame({'L.hat':pred_mean_x.summary_frame(alpha=0.05)['obs_ci_lower'],
              'U.hat':pred_mean_x.summary_frame(alpha=0.05)['obs_ci_upper']})
